In [ ]:
import sys
!{sys.executable} -m pip install --pre mesa
!{sys.executable} -m pip install colorama
!{sys.executable} -m pip install openpyxl
!{sys.executable} -m pip install --pre 'mesa[viz]'

In [11]:
# import libraries
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.animation import FuncAnimation
import mesa
import sys
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
import numpy as np
import pandas as pd
import math
from IPython.display import HTML
from matplotlib import rc
from matplotlib.colors import ListedColormap
from colorama import Fore, Back, Style


This code runs with mesa version 3.00 or higher

In [ ]:
# check mesa version
print("Mesa version: ", mesa.__version__)

In [ ]:
# Load the Excel file
#Floorplan_base_line.xlsx contains no signs; Floorplan_signs.xlsx contains signs throughout the floorplan and responds to intervention 1

file_path = '/Users/kimvanbruggen/Floorplan_base_line.xlsx'  # Replace with your file path
df = pd.read_excel(file_path, header=None)  # Load the data without assuming any headers

In [ ]:
# Initialize lists for each category
self_obstacles = []
self_empty = []
self_exits = []
self_signs = []
self_low_obstacles = []


# Iterate through the DataFrame to find the coordinates for each category
for y in range(df.shape[0]):  # Iterate over rows (height)
    for x in range(df.shape[1]):  # Iterate over columns (width)
        value = df.iloc[y, x]  # Get the value at position (x, y)
        
        # Check the value and store the coordinates in the appropriate list
        if value == 1:
            self_obstacles.append((x, y))
        elif value == 3:
            self_exits.append((x, y))
        elif value == 4:
            self_signs.append((x, y))
        elif value == 5:
            self_low_obstacles.append((x, y))
        elif value == 0:
            self_empty.append((x, y))

# Example print statements to check the lists
print("Obstacles:", self_obstacles)
print("Exits:", self_exits)
print("Signs:", self_signs)
print("low obstacles:", self_low_obstacles)	
print("Floor surface:", len(self_empty), "m2")

In [21]:
# Function to calculate Euclidean distance (distance between two points)
def euclidean_distance(pos1, pos2):
    # Use the formula to calculate distance between two coordinates
    return math.sqrt((pos1[0] - pos2[0]) ** 2 + (pos1[1] - pos2[1]) ** 2)

# Base Model

In [ ]:
###################
### AGENT CLASS ###
###################

class NavigationAgent(Agent):
    def __init__(self, model, vision=8, activated=True, toxicated=False):  # Default vision range of 8 cells
        super().__init__(model)  # MESA `Agent` class initialization
        self.found_exit = False  # Track if agent has reached the exit
        self.previous_pos = None  # Previous position of the agent
        self.vision = vision  # Vision range of the agent
        self.activated = activated  # Activation state of the agent
        self.toxicated = toxicated  # Toxicated state of the agent
        self.memory = set()  # Memory to keep track of seen signs
        self.step_count = 0  # Track the number of steps taken by the agent

    def step(self):
        self.step_count += 1
        if not self.activated:
            vision_area = self.get_filtered_vision_area()
            activated_neighbors = 0
            for neighbor_pos in vision_area:
                neighbor_agents = self.model.grid.get_cell_list_contents(neighbor_pos)
                for agent in neighbor_agents:
                    if isinstance(agent, NavigationAgent) and agent.activated:
                        activated_neighbors += 1
                        if (not self.toxicated and activated_neighbors >= 1) or (self.toxicated and activated_neighbors >= 3):
                            self.activated = True
                            break
                if self.activated:
                    break
            if not self.activated:
                return
        
        if self.pos in self.model.exit_locations:
            self.found_exit = True
            self.model.grid.remove_agent(self)
            self.remove()
            self.model.cumulative_exited += 1
        else:
            vision_area = self.get_filtered_vision_area()
            exits_in_vision = [exit_loc for exit_loc in self.model.exit_locations if exit_loc in vision_area]
            signs_in_vision = [sign_loc for sign_loc in vision_area if sign_loc in self.model.signs and sign_loc not in self.memory]
            if exits_in_vision:
                self.move_towards_target(exits_in_vision)
            elif signs_in_vision:
                self.move_towards_target(signs_in_vision)
                self.memory.update(signs_in_vision)
            else:
                self.move_randomly()

    # Function to get the agent's vision area, excluding cells blocked by obstacles
    def get_filtered_vision_area(self):
        vision_area = set()
        x, y = self.pos
        for dx in range(-self.vision, self.vision + 1):
            for dy in range(-self.vision, self.vision + 1):
                nx, ny = x + dx, y + dy
                if not (0 <= nx < self.model.grid.width and 0 <= ny < self.model.grid.height):
                    continue
                if not self.is_visible((nx, ny)):
                    continue
                vision_area.add((nx, ny))
        return vision_area

    # Helper function to check if a target cell is visible considering obstacles
    def is_visible(self, target):
        x1, y1 = self.pos
        x2, y2 = target
        dx, dy = abs(x2 - x1), abs(y2 - y1)
        sx, sy = (1 if x1 < x2 else -1), (1 if y1 < y2 else -1)
        err = dx - dy

        while (x1, y1) != (x2, y2):
            if (x1, y1) in self.model.obstacles:
                return False
            e2 = 2 * err
            if e2 > -dy:
                err -= dy
                x1 += sx
            if e2 < dx:
                err += dx
                y1 += sy
        return True

    # Function to move the agent randomly if no target is in sight
    def move_randomly(self):
        self.previous_pos = self.pos
        possible_steps = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        valid_steps = [step for step in possible_steps if step not in self.model.obstacles and step not in self.model.low_obstacles and len(self.model.grid.get_cell_list_contents(step)) < 8]
        if valid_steps:
            random_step = self.random.choice(valid_steps)
            self.model.grid.move_agent(self, random_step)

    # Function to move the agent towards a target (exit or sign)
    def move_towards_target(self, target_locations):
        self.previous_pos = self.pos
        possible_steps = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        min_distance = float('inf')
        best_step = None
        for step in possible_steps:
            if step not in self.model.obstacles and step not in self.model.low_obstacles and len(self.model.grid.get_cell_list_contents(step)) < 8:
                dist_to_closest_target = min(euclidean_distance(step, target_loc) for target_loc in target_locations)
                if dist_to_closest_target < min_distance:
                    min_distance = dist_to_closest_target
                    best_step = step
        if best_step:
            self.model.grid.move_agent(self, best_step)

    # Define the actions the agent will take in each step
    def step(self):
        if not self.activated:
            vision_area = self.get_filtered_vision_area()
            activated_neighbors = 0
            for neighbor_pos in vision_area:
                neighbor_agents = self.model.grid.get_cell_list_contents(neighbor_pos)
                for agent in neighbor_agents:
                    if isinstance(agent, NavigationAgent) and agent.activated:
                        activated_neighbors += 1
                        if (not self.toxicated and activated_neighbors >= 1) or (self.toxicated and activated_neighbors >= 3):
                            self.activated = True
                            break
                if self.activated:
                    break
            if not self.activated:
                return
        if self.pos in self.model.exit_locations:
            self.found_exit = True
            self.model.grid.remove_agent(self)
            self.remove()
            self.model.cumulative_exited += 1
        else:
            vision_area = self.get_filtered_vision_area()
            exits_in_vision = [exit_loc for exit_loc in self.model.exit_locations if exit_loc in vision_area]
            signs_in_vision = [sign_loc for sign_loc in vision_area if sign_loc in self.model.signs and sign_loc not in self.memory]
            if exits_in_vision:
                self.move_towards_target(exits_in_vision)
            elif signs_in_vision:
                self.move_towards_target(signs_in_vision)
                self.memory.update(signs_in_vision)
            else:
                self.move_randomly()

###################
### MODEL CLASS ###
###################

# Define the model class to handle the overall environment
class FloorPlanModel(Model):
    def __init__(self, width, height, num_agents, agent_vision):
        super().__init__()  # `Model` class initialization
        
        # Basic model settings
        self.num_agents = num_agents
        self.agent_vision = agent_vision
        self.grid = MultiGrid(width, height, False)  # MESA grid with dimensions; False means no wrapping
        self.exit_locations = self_exits  # Place exits

        # Define obstacles and signs in the grid
        self.obstacles = self_obstacles
        self.signs = self_signs 
        self.low_obstacles = self_low_obstacles
        
        # Initialize cumulative exited count
        self.cumulative_exited = 0

        # Initialize DataCollector (MESA tool for tracking metrics across steps)
        self.datacollector = DataCollector(
            model_reporters={
                "Active Agents": lambda m: len(m.agents),  # Count of agents still active
                "Exited Agents": lambda m: sum(1 for agent in m.agents if isinstance(agent, NavigationAgent) and agent.found_exit),
                "Cumulative Exited Agents": lambda m: m.cumulative_exited,  # Cumulative exited count
                "Agents per Cell": self.count_agents_per_cell  # Counts agents in each cell
            },
            agent_reporters={
                "Found Exit": lambda a: a.found_exit if isinstance(a, NavigationAgent) else None,  # Reports exit status per agent
                "Toxicated": lambda a: a.toxicated if isinstance(a, NavigationAgent) else None,  # Reports toxicated status per agent
                "Step Count": lambda a: a.step_count if isinstance(a, NavigationAgent) else None  # Reports step count per agent
            }
        )

        self.place_agents(agent_vision)  # Place agents on the grid
        self.datacollector.collect(self)  # Collect data at the start of the simulation

    # Function to randomly place agents in the grid
    def place_agents(self, agent_vision):
        num_activated_agents = int(self.num_agents * 0.40)  # Calculate 30% of the total agents
        num_toxicated_agents = int(self.num_agents * 0.30)  # Calculate 60% of the total agents
        for i in range(self.num_agents):
            activated = i < num_activated_agents  # Activate the first 30% of agents
            toxicated = i < num_toxicated_agents  # Toxicate the first 60% of agents
            vision = agent_vision if not toxicated else agent_vision // 2  # Set vision based on toxicated state
            agent = NavigationAgent(self, vision=vision, activated=activated, toxicated=toxicated)  # Create agent with initial vision range, activation state, and toxicated state
            placed = False  # Track if the agent is successfully placed
            while not placed:
                x = self.random.randrange(self.grid.width)
                y = self.random.randrange(self.grid.height)
                cell_contents = self.grid.get_cell_list_contents((x, y))

                # Only place agent if cell has no obstacles, low obstacles, and fewer than 8 agents
                if (x, y) not in self.obstacles and (x, y) not in self.low_obstacles and len(cell_contents) < 8:
                    self.grid.place_agent(agent, (x, y))  # MESA function to place agent in grid
                    placed = True  # Mark as placed

    # Function to count agents in each cell
    def count_agents_per_cell(self):
        agent_counts = {}  # Dictionary to store agent counts by cell position
        # MESA `coord_iter` function iterates over grid cells and their contents
        for cell in self.grid.coord_iter():
            cell_contents, (x, y) = cell  # Unpack cell contents and coordinates
            # Count NavigationAgents in each cell
            nav_agent_count = sum(1 for obj in cell_contents if isinstance(obj, NavigationAgent))
            if nav_agent_count > 0:
                agent_counts[(x, y)] = nav_agent_count
        return agent_counts
    
    # Function to get the grid data for visualization
    def get_grid(self):
        # 0: empty, 1: obstacle, 2: agent, 3: exit, 4: sign, 5: low obstacle
        grid_data = np.zeros((self.grid.height, self.grid.width)) # Adjust order to (height, width)
        
        # Mark obstacles on the grid
        for x, y in self.obstacles:
            grid_data[y, x] = 1
        
        # Mark agents on the grid
        for agent in self.agents:
            if isinstance(agent, NavigationAgent):
                x, y = agent.pos
                grid_data[y, x] = 2
        
        # Mark signs and exit on the grid
        for x, y in self.signs:
            grid_data[y, x] = 4
        for x, y in self.exit_locations:
            grid_data[y, x] = 3  

        # Mark low obstacles on the grid
        for x, y in self.low_obstacles:
            grid_data[y, x] = 5

        return grid_data

    # Model step function to update the simulation
    def step(self):
        self.agents.do("step")  # MESA 3.0 function to execute the `step` function of each agent
        self.datacollector.collect(self)  # MESA DataCollector collects metrics at each step

# ##### Batch running ######

# def run_batch(model_class, grid_width, grid_height, num_agents, vision, num_steps, num_runs, lights, signs, intoxicated_percentage):
#     toxicated_exit_times = []
#     sober_exit_times = []
#     avg_exit_times = []

#     for _ in range(num_runs):
#         model = model_class(grid_width, grid_height, num_agents, vision)
#         for _ in range(num_steps):
#             model.step()

#         agent_data = model.datacollector.get_agent_vars_dataframe()
#         toxicated = agent_data[agent_data["Toxicated"] == True]["Step Count"].dropna()
#         sober = agent_data[agent_data["Toxicated"] == False]["Step Count"].dropna()

#         toxicated_exit_times.append(toxicated.mean() if not toxicated.empty else float('inf'))
#         sober_exit_times.append(sober.mean() if not sober.empty else float('inf'))
#         avg_exit_times.append(agent_data["Step Count"].mean())

#     return toxicated_exit_times, sober_exit_times, avg_exit_times

# ##### Matrix ######

# def run_all_scenarios():
#     grid_width, grid_height = 34, 20
#     num_agents = 100
#     vision = 6
#     num_steps = 300
#     num_runs = 30

#     scenarios = [
#         {"lights": False, "signs": False, "intoxicated_percentage": 0.0},
#         {"lights": True, "signs": False, "intoxicated_percentage": 0.0},
#         {"lights": False, "signs": True, "intoxicated_percentage": 0.0},
#         {"lights": True, "signs": True, "intoxicated_percentage": 0.0},
#         {"lights": False, "signs": False, "intoxicated_percentage": 0.3},
#         {"lights": True, "signs": False, "intoxicated_percentage": 0.3},
#         {"lights": False, "signs": True, "intoxicated_percentage": 0.3},
#         {"lights": True, "signs": True, "intoxicated_percentage": 0.3},
#         {"lights": False, "signs": False, "intoxicated_percentage": 0.6},
#         {"lights": True, "signs": False, "intoxicated_percentage": 0.6},
#         {"lights": False, "signs": True, "intoxicated_percentage": 0.6},
#         {"lights": True, "signs": True, "intoxicated_percentage": 0.6},
#     ]

#     results = []

#     for scenario in scenarios:
#         toxicated_exit_times, sober_exit_times, avg_exit_times = run_batch(
#             FloorPlanModel, grid_width, grid_height, num_agents, vision, num_steps, num_runs,
#             scenario["lights"], scenario["signs"], scenario["intoxicated_percentage"]
#         )
#         results.append({
#             "lights": scenario["lights"],
#             "signs": scenario["signs"],
#             "intoxicated_percentage": scenario["intoxicated_percentage"],
#             "toxicated_exit_times": toxicated_exit_times,
#             "sober_exit_times": sober_exit_times,
#             "avg_exit_times": avg_exit_times
#         })

#     return results

# def print_results(results):
#     for result in results:
#         print(f"Lights: {result['lights']}, Signs: {result['signs']}, Intoxicated percentage: {result['intoxicated_percentage']}")
#         print(f"  Average intoxicated exit time: {np.mean(result['toxicated_exit_times'])}")
#         print(f"  Average sober exit time: {np.mean(result['sober_exit_times'])}")
#         print(f"  Average exit time: {np.mean(result['avg_exit_times'])}")
#         print()

# results = run_all_scenarios()
# print_results(results)

###### Graph with exit times ######

# import matplotlib.pyplot as plt

# grid_width, grid_height = 34, 20
# num_agents = 50
# vision = 6
# num_steps = 300
# num_runs = 20

# toxicated_exit_times, sober_exit_times, avg_exit_times = run_batch(
#     FloorPlanModel, grid_width, grid_height, num_agents, vision, num_steps, num_runs
# )

# print("Toxicated exit times:", toxicated_exit_times)
# print("Sober exit times:", sober_exit_times)
# print("Average exit times:", avg_exit_times)

# plt.figure(figsize=(10, 6))
# plt.plot(toxicated_exit_times, label="Intoxicated agents exit time", color="red")
# plt.plot(sober_exit_times, label="Sober agents exit time", color="blue")
# plt.plot(avg_exit_times, label="Average exit time", color="green")
# plt.xlabel("Simulation run")
# plt.ylabel("Exit time (steps)")
# plt.title("Exit times: intoxicated vs sober agents")
# plt.legend()
# plt.show()

In [23]:
#grid_data = model.get_grid()

In [ ]:
# Run the model and see the results
model = FloorPlanModel(34, 20, 300, 6) # run the model with 34x20 grid and 100 agents and a vision of 6
for i in range(300): # run the model for 100 steps
    model.step() # step the model by 1

# Collect the data from the model
model_data = model.datacollector.get_model_vars_dataframe()
agent_data = model.datacollector.get_agent_vars_dataframe()

grid_data = model.get_grid()

In [ ]:
model_data # print the model data

In [ ]:
agent_data # print the agent data

# Visualization with User Interface
You don't have to understand every single line in the visualisation code below. Please understand the code to extend that you can introduce changes to it when needed

In [ ]:
# Visualization Function
def plot_grid(model, ax):
    rc("animation", embed_limit=100)  # Set a higher limit in MB to allow smoother animation playback
    grid_data = model.get_grid()  # Retrieve the current state of the grid from the model
    ax.clear()  # Clear any previous plots on the axes to prevent overlap in visualizations
    
    # Define color mappings:
    # 0 (empty) -> white, 1 (obstacle) -> black, 2 (agent) -> blue,
    # 3 (exit) -> red, 4 (sign) -> yellow, 5 (low_obstacle) -> purple
    cmap = mcolors.ListedColormap(['white', 'black', 'blue', 'red', 'yellow', 'purple'])
    bounds = [0, 1, 2, 3, 4, 5, 6]  # Boundaries to separate each category
    norm = mcolors.BoundaryNorm(bounds, cmap.N)  # Normalizes values to assign colors to each category
    
    # Display the grid data with color mapping applied.
    # Setting 'origin' to 'lower' places the (0,0) coordinate at the bottom-left.
    ax.imshow(grid_data, cmap=cmap, norm=norm, origin='lower')
    
    # Add grid lines for better cell visibility
    ax.grid(which='both', color='gray', linestyle='-', linewidth=2)
    
    # Customize grid display: set grid to start from -0.5 with labels at intervals of 1
    # Set minor ticks for cell boundaries, with thicker and darker lines
    ax.set_xticks(np.arange(-0.5, model.grid.width, 1), minor=True)
    ax.set_yticks(np.arange(-0.5, model.grid.height, 1), minor=True)
    ax.grid(which='minor', color='gray', linestyle='-', linewidth=1.5)  # Thicker, darker lines for cell boundaries

    # Set major ticks for labels at intervals of 5, with lighter and thinner lines
    ax.set_xticks(np.arange(0, model.grid.width, 5), minor=False)
    ax.set_yticks(np.arange(0, model.grid.height, 5), minor=False)
    ax.grid(which='major', color='lightgray', linestyle='-', linewidth=0)  # invisible line

    
    # Label the exits on the grid in red text for easy identification
    for exit_x, exit_y in model.exit_locations:
        ax.text(exit_x, exit_y, 'EXIT', ha='center', va='center', fontsize=12, color='black', fontweight='bold')

    # Add labels for each sign location in black text for visibility
    for sign_pos in model.signs:
        x, y = sign_pos
        ax.text(x, y, 'SIGN', ha='center', va='center', fontsize=10, color='black', fontweight='bold')

    # Set the title to show the current step number in the model
    ax.set_title(f"Step {model.steps}", fontsize=16)
    ax.tick_params(axis='both', which='major', labelsize=7)  # Control tick label size
    
    # Draw movement arrows for agents to show the direction they are traveling
    for agent in model.agents:
        # Only draw arrow if the agent has moved and is a NavigationAgent
        if isinstance(agent, NavigationAgent) and agent.previous_pos:
            start_x, start_y = agent.previous_pos  # Previous position of the agent
            end_x, end_y = agent.pos  # Current position of the agent
            
            # Draw an arrow from the previous position to the current position
            ax.arrow(
                start_x, start_y,
                end_x - start_x, end_y - start_y,
                head_width=0.3, head_length=0.3, fc='yellow', ec='yellow'  # Yellow arrow for movement direction
            )

# Animation Update Function
def update(frame, model, ax):
    # For every frame, update the model state if it's not the first frame
    if frame > 0:
        model.step()  # Run one step of the model simulation
    plot_grid(model, ax)  # Redraw the grid with updated agent positions
    
# Run the Animation with a larger figure size
def run_animation(model, steps):
    fig, ax = plt.subplots(figsize=(5, 5))  # Create a 10x10 figure for the plot
    plot_grid(model, ax)  # Plot the initial grid state
    # Create an animation that updates the grid for each step
    anim = FuncAnimation(fig, update, frames=steps+1, fargs=(model, ax), repeat=False)
    plt.close(fig)  # Close the figure after animation creation to avoid duplicate displays
    return anim

# Your imports and function definitions remain the same, up until where you initialize and run the model
import ipywidgets as widgets  # For interactive widgets (slider, button)
from IPython.display import display, HTML  # To display widgets and HTML animations
import time  # For tracking elapsed time

# Slider to choose the number of agents
agent_slider = widgets.IntSlider(
    value=50,      # Default starting number of agents
    min=1,        # Minimum number of agents allowed
    max=500,       # Maximum number of agents allowed
    step=10,       # Step size for slider increments
    description='Num Agents:',  # Label for the slider
    continuous_update=False     # Only update value when slider is released
)

# Slider to choose the number of time steps (frames) for the animation
time_step_slider = widgets.IntSlider(
    value=50,      # Default starting number of steps
    min=1,         # Minimum time steps allowed
    max=500,       # Maximum time steps allowed
    step=1,        # Step size for slider increments
    description='Time Steps:',  # Label for the slider
    continuous_update=False     # Only update value when slider is released
)

# Slider to choose the vision range (vision radius) of agents
vision_slider = widgets.IntSlider(
    value=6,      # Default starting vision range
    min=1,        # Minimum vision range
    max=20,       # Maximum vision range
    step=1,       # Step size for slider increments
    description='Vision:',  # Label for the slider
    continuous_update=False     # Only update value when slider is released
)

# Button to start the simulation with current slider settings
run_button = widgets.Button(description="Run Simulation")

# Output widget for displaying the animation and elapsed time
output_widget = widgets.Output()

# Label to show elapsed time during the simulation
elapsed_time_label = widgets.Label(value="Elapsed time: 0.0 seconds")

# Flag variable to control timing function
stop_timer = False

# Display the interface with sliders, button, label, and output area
display(agent_slider, time_step_slider, vision_slider, run_button, elapsed_time_label, output_widget)

# Define the function to initialize and run the model
def run_model(change):
    global stop_timer, model_data, agent_data  # Allow variables to be accessed globally
    with output_widget:  # Use output widget to display the animation
        output_widget.clear_output()  # Clear any previous output
        num_agents = agent_slider.value  # Get the number of agents from the slider
        time_steps = time_step_slider.value  # Get the number of steps from the slider
        agent_vision = vision_slider.value  # Get the vision range from the slider
        model = FloorPlanModel(width=34, height=20, num_agents=num_agents, agent_vision=agent_vision)  # Initialize the model


        # Reset timer flag and start timing for the animation
        stop_timer = False
        start_time = time.time()

        def update_time_label():
            while not stop_timer:  # Keep updating time label until timer stops
                elapsed_time = time.time() - start_time
                elapsed_time_label.value = f"Elapsed time: {elapsed_time:.1f} seconds"
                time.sleep(0.1)  # Update every 0.1 seconds for real-time effect

        # Start elapsed time tracking in a separate thread
        import threading
        timer_thread = threading.Thread(target=update_time_label, daemon=True)
        timer_thread.start()

        # Run the animation with selected number of steps
        anim = run_animation(model, steps=time_steps)

        # Display the animation output in HTML format
        output = HTML(anim.to_jshtml())
        display(output)

        # Stop the timer after the simulation completes
        stop_timer = True
        elapsed_time = time.time() - start_time
        elapsed_time_label.value = f"Total elapsed time: {elapsed_time:.1f} seconds"

        # Retrieve and display model and agent data after simulation completes
        model_data = model.datacollector.get_model_vars_dataframe()  # Data for the model over time
        agent_data = model.datacollector.get_agent_vars_dataframe()  # Data for each agent over time
    return model, model_data, agent_data  # Return model and data for further inspection

# Attach the run_model function to the run button click event
run_button.on_click(run_model)


In [ ]:
def visualize_static_floorplan(model):
    # Get the grid data from the model (static view, no agents)
    grid_data = model.get_grid()

    # Create a colormap with numerical values for each cell type
    # 0: empty (white), 1: obstacle (black), 2: (agent), 3: exit (red), 4: sign (yellow), 5: low obstacle (purple)
    cmap = ListedColormap(["white", "black", "white", "red", "yellow", "purple"])
    norm = plt.Normalize(vmin=0, vmax=5)

    # Visualize the floorplan
    plt.figure(figsize=(8, 8))
    plt.imshow(grid_data, cmap=cmap, norm=norm)

    for y in range(grid_data.shape[0]):
        for x in range(grid_data.shape[1]):
            if grid_data[y, x] == 3:  # Exit
                plt.text(x, y, 'Exit', color='black', ha='center', va='center', fontsize=8, weight='bold')
            elif grid_data[y, x] == 4:  # Sign
                plt.text(x, y, 'Sign', color='black', ha='center', va='center', fontsize=8, weight='bold')

    plt.title("Static Floorplan")
    plt.axis("off")  # Remove axis for a cleaner look
    plt.show()

# Usage: Assuming the model is already created
visualize_static_floorplan(model)

In [ ]:
print("\nModel Data:")
model_data

In [ ]:
print("\nAgent Data:")
agent_data.head(40)


In [ ]:
# Uncomment the code below to display the 'Agents per Cell' data for each step

print("Agents per Cell at each step:")
for step, agents_per_cell in model_data["Agents per Cell"].items():
   print(f"\nStep {step}:")
   for cell, count in agents_per_cell.items():
       print(f"  Cell {cell}: {count} agent(s)")